In [1]:
import time
import os
import gc

import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.utils import shuffle, resample
from sklearn.metrics import mean_squared_error, r2_score
np.set_printoptions(precision=6, suppress=True)

import PIL
from PIL import Image

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
tf.__version__

'2.3.0'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


# Hyperparameters

In [4]:
BEST_PATH = './models/pg_convnet_test.h5'
TRAINING_EPOCHS = 200
LEARNING_RATE = 0.001
EPSILON = 1e-08
BATCH_SIZE = 128

# Data preparation

In [5]:
l = np.load('./results/pg_dataset.npz', allow_pickle=True)
data_indices_2020S = l['data_indices_2020S']
output_label_2020S = l['output_label_2020S']
data_indices_2020W = l['data_indices_2020W']
output_label_2020W = l['output_label_2020W']
OUTPUT_MAXS = l['OUTPUT_MAXS']
OUTPUT_MINS = l['OUTPUT_MINS']

In [6]:
COLUMNS = ['node_len', 'node_dia', 'plant_h', 'leaf_area']

In [7]:
output_2020S_df = pd.DataFrame(output_label_2020S, index=data_indices_2020S, columns=COLUMNS)
output_2020W_df = pd.DataFrame(output_label_2020W, index=data_indices_2020W, columns=COLUMNS)

In [8]:
DIRECTORY = './images/2020_S/'
dir_list = os.listdir(DIRECTORY)
dataset_list = [direc for direc in dir_list if direc.startswith('LAI_OVER') or direc.startswith('LAI2_OVER')]
dataset_list.sort()

In [9]:
data_indices = []
input_images = []
output_labels = []
for DATE in output_2020S_df.index:
    for DIRECTORY in dataset_list:
        if DATE in DIRECTORY:
            file_list = os.listdir(f'./images/2020_S/{DIRECTORY}')
            file_list = [file for file in file_list if file.endswith('.jpg')]
            for FILE in file_list:
                TIME = pd.Timedelta(FILE.split()[-1].split('.')[0])
                if TIME >= pd.Timedelta('08:00:00') and TIME <= pd.Timedelta('16:00:00'):
                    image = Image.open(f'./images/2020_S/{DIRECTORY}/{FILE}')
                    data_indices.append(pd.Timestamp(DATE))
                    input_images.append(img_to_array(image))
                    output_labels.append(output_2020S_df.loc[DATE].values)

In [10]:
input_images = np.stack(input_images, axis=0)
output_labels = np.stack(output_labels, axis=0)
data_indices = np.array(data_indices)

In [11]:
print(len(data_indices))
print(input_images.shape)
print(output_labels.shape)

6392
(6392, 256, 256, 3)
(6392, 4)


In [12]:
gc.collect()

38

In [13]:
data_indices, input_images, output_labels = shuffle(data_indices, input_images, output_labels, random_state=4574)

In [14]:
N_TRAIN = int(input_images.shape[0]*.7)
train_input = input_images[:N_TRAIN, ...]
train_label = output_labels[:N_TRAIN, ...]
train_index = data_indices[:N_TRAIN]
val_input = input_images[N_TRAIN:, ...]
val_label = output_labels[N_TRAIN:, ...]
val_index = data_indices[N_TRAIN:]

In [15]:
datagen = ImageDataGenerator(
    rescale=1/255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [16]:
train_iterator = datagen.flow(train_input, train_label, batch_size=BATCH_SIZE)
val_iterator = datagen.flow(val_input, val_label, batch_size=BATCH_SIZE)

In [17]:
print(f'number of training set: {input_images.shape[0]}')
print(f'number of validation set: {val_input.shape[0]}')

number of training set: 6392
number of validation set: 1918


In [18]:
gc.collect()

0

# Model construction

In [19]:
class ResidualBlock(layers.Layer):
    def __init__(self, num_filter, stride=1):
        super(ResidualBlock, self).__init__()
        self.n = num_filter
        self.s = stride
        
        self.conv1 = layers.Conv2D(filters=self.n, kernel_size=1, strides=1, kernel_initializer='glorot_normal', padding='same')
        self.norm1 = layers.BatchNormalization()
        self.act1 = layers.Activation(activations.relu)
        self.conv2 = layers.Conv2D(filters=self.n, kernel_size=3, strides=stride, kernel_initializer='glorot_normal', padding='same')
        self.norm2 = layers.BatchNormalization()
        self.act2 = layers.Activation(activations.relu)
        self.conv3 = layers.Conv2D(filters=self.n*4, kernel_size=1, strides=1, kernel_initializer='glorot_normal', padding='same')
        self.norm3 = layers.BatchNormalization()
        self.act3 = layers.Activation(activations.relu)

        self.downsample = Sequential()
        self.downsample.add(layers.Conv2D(filters=self.n*4, kernel_size=1, strides=stride, kernel_initializer='glorot_normal'))
        self.downsample.add(layers.BatchNormalization())

    def call(self, inp, training=None, **kwargs):
        shortcut = self.downsample(inp)

        inp = self.act1(self.norm1(self.conv1(inp), training=training))
        inp = self.act2(self.norm2(self.conv2(inp), training=training))
        inp = self.norm3(self.conv3(inp), training=training)

        oup = self.act3(layers.add([shortcut, inp]))

        return oup

In [20]:
def block_maker(num_filter, num_blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(ResidualBlock(num_filter, stride=stride))

    for _ in range(1, num_blocks):
        res_block.add(ResidualBlock(num_filter, stride=1))

    return res_block

In [21]:
class ResNet2D(Model):
    def __init__(self, layer_params):
        super(ResNet2D, self).__init__()
        self.conv1 = layers.Conv2D(filters=64, kernel_size=7, strides=2, kernel_initializer='glorot_normal', padding="same")
        self.norm1 = layers.BatchNormalization()
        self.act1 = layers.Activation(activations.relu)
        self.pool1 = layers.MaxPool2D(pool_size=(3, 3), strides=2, padding="same")

        self.layer1 = block_maker(num_filter=64, num_blocks=layer_params[0])
        self.layer2 = block_maker(num_filter=128, num_blocks=layer_params[1], stride=2)
        self.layer3 = block_maker(num_filter=256, num_blocks=layer_params[2], stride=2)
        self.layer4 = block_maker(num_filter=512, num_blocks=layer_params[3], stride=2)

        self.flat = layers.Flatten()
        self.dense1 = layers.Dense(units=512, activation='relu')
        self.dense2 = layers.Dense(units=128, activation='relu')
        self.fc = layers.Dense(units=4)

    def call(self, inp, training=None, mask=None):
        inp = self.conv1(inp)
        inp = self.norm1(inp, training=training)
        inp = self.act1(inp)
        inp = self.pool1(inp)
        inp = self.layer1(inp, training=training)
        inp = self.layer2(inp, training=training)
        inp = self.layer3(inp, training=training)
        inp = self.layer4(inp, training=training)
        inp = self.dense1(self.flat(inp))
        inp = self.dense2(inp)
        output = self.fc(inp)

        return output

In [22]:
with strategy.scope():
    model = ResNet2D(layer_params=[2, 3, 2, 1])

In [23]:
cbs = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=.1, patience=5, verbose=0, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0)

save = callbacks.ModelCheckpoint(
    BEST_PATH, monitor='val_loss', verbose=0,
    save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')

early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20) 

In [24]:
with strategy.scope():
    opt = optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    model.compile(optimizer=opt, loss='mae')
    model.fit(train_iterator, epochs=TRAINING_EPOCHS, validation_data=val_iterator,
                  verbose=1, callbacks=[cbs, save, early_stop]) 

Epoch 1/200
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
35/35 [==============================] - ETA: 0s - loss: 6.7070INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INF

KeyboardInterrupt: 

In [ ]:
# model.summary()

In [ ]:
model.load_weights(BEST_PATH)

In [ ]:
model.evaluate(val_iterator)

0.013615048490464687

0.016493843868374825 / 1211

In [ ]:
del(train_index)
del(train_input)
del(train_iterator)
del(train_label)

In [ ]:
gc.collect()

In [ ]:
pred_result = model.predict(val_input/255)

In [ ]:
print(pred_result.shape)
print(val_label.shape)

In [ ]:
val_label = (OUTPUT_MAXS - OUTPUT_MINS)*val_label + OUTPUT_MINS
pred_result = (OUTPUT_MAXS - OUTPUT_MINS)*pred_result + OUTPUT_MINS

In [ ]:
pred_df = pd.DataFrame(np.concatenate([val_label, pred_result], axis=1), index=val_index)
pred_df.columns = ['label_node_len', 'label_node_dia', 'label_plant_h', 'label_leaf_area', 'pred_node_len', 'pred_node_dia', 'pred_plant_h', 'pred_leaf_area']

In [ ]:
pred_df.to_csv('./results/model_output/pg_pred_result_test.csv')